In [17]:
ranking.iloc[:10]

,degree_centrality_out,rank,label
Klf12,0.262278,1.0,False
Rora,0.070010,2.0,False
Mef2a,0.054336,3.0,True
Thrb,0.044932,4.0,False
Mef2c,0.042842,5.0,True
Mlxipl,0.041797,6.0,False
E2f3,0.032393,7.5,False
Foxn3,0.032393,7.5,False
Rbpj,0.030303,10.0,False
Ebf3,0.030303,10.0,False


In [1]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score, f1_score

In [2]:
true_labels = pd.read_csv("/cellar/users/aklie/projects/igvf/topic_grn_links/data/refs/microglia_regulators.csv")
true_labels

,section,regulator
0,elucidating_critical_transcriptional_regulator...,Spi1
1,elucidating_critical_transcriptional_regulator...,Sall1
2,elucidating_critical_transcriptional_regulator...,Mafb
3,elucidating_critical_transcriptional_regulator...,Mef2c
4,elucidating_critical_transcriptional_regulator...,Stat3
5,elucidating_critical_transcriptional_regulator...,Usf1
6,elucidating_critical_transcriptional_regulator...,Smad2
7,table_1,Runx1
8,table_1,Irf8
9,table_1,Mef2a


In [12]:
ranking = pd.read_csv("/cellar/users/aklie/projects/igvf/topic_grn_links/eval/results/igvf_b01_LeftCortex/01Sep23/tf_rankings/celloracle_balanced_genotype_microglia_both_0.05_log1p_cp10k/degree_centrality_out.tsv", sep="\t", index_col=0)
ranking

,degree_centrality_out,rank
Klf12,0.262278,1.0
Rora,0.070010,2.0
Mef2a,0.054336,3.0
Thrb,0.044932,4.0
Mef2c,0.042842,5.0
...,...,...
Arhgef12,0.001045,173.0
Tfcp2,0.001045,173.0
Ikzf2,0.001045,173.0
Mbtps2,0.001045,173.0


In [13]:
ranking["label"] = ranking.index.isin(true_labels["regulator"])

In [15]:
# Extract scores and true labels from the ranking
scores = ranking['degree_centrality_out'].values
y_true = ranking['label'].values

# Compute Precision-Recall curve values
precision, recall, thresholds = precision_recall_curve(y_true, scores)

# Compute the Area Under the PR Curve (AUPR)
aupr = auc(recall, precision)

# Compute the Area Under the ROC Curve (AUROC)
auroc = roc_auc_score(y_true, scores)

# Compute F1 scores across the thresholds
f1_scores = 2 * (precision * recall) / (precision + recall)
best_f1 = max(f1_scores)
best_threshold = thresholds[f1_scores.argmax()]

print(f"Area Under the PR Curve (AUPR): {aupr}")
print(f"Area Under the ROC Curve (AUROC): {auroc}")
print(f"Best F1 Score: {best_f1} at threshold: {best_threshold}")

Area Under the PR Curve (AUPR): 0.10378887414321188
Area Under the ROC Curve (AUROC): 0.6068786982248521
Best F1 Score: 0.3076923076923077 at threshold: 0.0700104493207941


/tmp/ipykernel_1252200/1408779247.py:15: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)


In [16]:
# Shuffle the ranking and re-compute the metrics
np.random.seed(42)
ranking_shuffled = ranking.sample(frac=1)
aupr_shuffled = auc(recall, precision)
auroc_shuffled = roc_auc_score(y_true, scores)
f1_scores_shuffled = 2 * (precision * recall) / (precision + recall)
best_f1_shuffled = max(f1_scores_shuffled)
best_threshold_shuffled = thresholds[f1_scores_shuffled.argmax()]

print(f"Shuffled Area Under the PR Curve (AUPR): {aupr_shuffled}")
print(f"Shuffled Area Under the ROC Curve (AUROC): {auroc_shuffled}")
print(f"Shuffled Best F1 Score: {best_f1_shuffled} at threshold: {best_threshold_shuffled}")

Shuffled Area Under the PR Curve (AUPR): 0.10378887414321188
Shuffled Area Under the ROC Curve (AUROC): 0.6068786982248521
Shuffled Best F1 Score: 0.3076923076923077 at threshold: 0.0700104493207941


/tmp/ipykernel_1252200/1160163765.py:6: RuntimeWarning: invalid value encountered in divide
  f1_scores_shuffled = 2 * (precision * recall) / (precision + recall)


# TF ranking
Subset to only the TFs for all methods
- 